In [11]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

# Đọc dữ liệu từ file CSV
df = pd.read_csv('transactions.csv')

# Chuyển đổi dữ liệu thành định dạng của Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'product_id', 'rating']], reader)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
trainset, testset = train_test_split(data, test_size=0.2)

In [13]:
# Tạo mô hình SVD
model = SVD()

# Huấn luyện mô hình trên tập huấn luyện
model.fit(trainset)

In [14]:
# Tạo dự đoán trên tập kiểm tra
predictions = model.test(testset)

# Đánh giá mô hình
accuracy.rmse(predictions)

# Hiển thị kết quả dự đoán
print("Predictions:")
for uid, iid, true_r, est, _ in predictions[:10]:
    print(f'User {uid:<10} - Product {iid:<10} | Actual: {true_r:<5.2}, Predicted: {est:<5.2f}')

RMSE: 0.4498
Predictions:
User 23990699   - Product 160902809  | Actual: 4.0  , Predicted: 3.66 
User 27009952   - Product 164550314  | Actual: 3.4  , Predicted: 3.35 
User 17472712   - Product 102085482  | Actual: 5.0  , Predicted: 4.23 
User 26193430   - Product 68219838   | Actual: 3.0  , Predicted: 3.04 
User 17823354   - Product 164252148  | Actual: 2.6  , Predicted: 2.47 
User 21393370   - Product 196174372  | Actual: 3.6  , Predicted: 3.39 
User 1664714    - Product 58221635   | Actual: 2.0  , Predicted: 2.30 
User 15112955   - Product 103076103  | Actual: 3.5  , Predicted: 3.29 
User 5903960    - Product 75401284   | Actual: 2.3  , Predicted: 2.74 
User 5457362    - Product 19364741   | Actual: 2.2  , Predicted: 2.20 


In [4]:
# Hàm gợi ý sản phẩm
def get_top_n_recommendations(user_id, n=10):
    # Lấy danh sách tất cả các product_id
    all_product_ids = df['product_id'].unique()
    
    # Dự đoán rating cho tất cả các sản phẩm mà user chưa đánh giá
    user_ratings = [(product_id, model.predict(user_id, product_id).est) for product_id in all_product_ids]
    
    # Sắp xếp theo rating dự đoán
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    
    # Lấy top n sản phẩm
    top_n_recommendations = user_ratings[:n]
    
    return top_n_recommendations

In [5]:
# Ví dụ: Gợi ý 10 sản phẩm cho user_id = 1
recommendations = get_top_n_recommendations(22580465, 10)
print("\nTop Recommendations:")
for product_id, rating in recommendations:
    print(f'Product ID: {product_id:<10}, Predicted Rating: {rating:<5.2f}')


Top Recommendations:
Product ID: 14847948  , Predicted Rating: 4.40 
Product ID: 205712436 , Predicted Rating: 4.38 
Product ID: 130287356 , Predicted Rating: 4.36 
Product ID: 45480982  , Predicted Rating: 4.34 
Product ID: 107709913 , Predicted Rating: 4.31 
Product ID: 197536148 , Predicted Rating: 4.29 
Product ID: 52417141  , Predicted Rating: 4.26 
Product ID: 121009977 , Predicted Rating: 4.23 
Product ID: 29916656  , Predicted Rating: 4.19 
Product ID: 46698977  , Predicted Rating: 4.17 


In [12]:
df['rating']

0       3.5
1       4.0
2       3.4
3       1.0
4       1.7
       ... 
7071    3.2
7072    2.2
7073    2.8
7074    2.8
7075    2.6
Name: rating, Length: 7076, dtype: float64

In [15]:
df.product_id.nunique()

505

In [16]:
# Ví dụ: Gợi ý 10 sản phẩm cho user_id = 1
recommendations = get_top_n_recommendations(16877862, 10)
for product_id, rating in recommendations:
    print(f'Product ID: {product_id}, Predicted Rating: {rating}')

Product ID: 46640788, Predicted Rating: 4.586202392465683
Product ID: 107369821, Predicted Rating: 4.506402086325367
Product ID: 100179757, Predicted Rating: 4.494172949747698
Product ID: 45480982, Predicted Rating: 4.457981358846178
Product ID: 205712436, Predicted Rating: 4.453283030356523
Product ID: 275225073, Predicted Rating: 4.427522980451351
Product ID: 22543488, Predicted Rating: 4.422423968395485
Product ID: 46698977, Predicted Rating: 4.384747294761259
Product ID: 93710743, Predicted Rating: 4.381182876118729
Product ID: 134025169, Predicted Rating: 4.3545586279918025
